<a href="https://colab.research.google.com/github/beBijayeeni/StockAnalysisDashboard/blob/main/stock_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import pandas as pd
from openpyxl import load_workbook
import yfinance as yf
import plotly.graph_objects as go

from dash import Dash, dcc, html, dash_table, Input, Output, State

# ====================
# Excel Utility Function
# ====================

EXCEL_FILE = "stock_reports.xlsx"

def save_to_excel(data_list):
    """
    Save a list of data to an Excel file.
    data_list: [Ticker, Exchange, Period, High, Low]
    """
    columns = ["Ticker", "Exchange", "Period", "High", "Low"]

    if os.path.exists(EXCEL_FILE):
        try:
            df_existing = pd.read_excel(EXCEL_FILE)
            df_new = pd.DataFrame([data_list], columns=columns)
            df_result = pd.concat([df_existing, df_new], ignore_index=True)
        except Exception as e:
            print("Error reading existing Excel file:", e)
            df_result = pd.DataFrame([data_list], columns=columns)
    else:
        df_result = pd.DataFrame([data_list], columns=columns)

    df_result.to_excel(EXCEL_FILE, index=False)
    print(f"Data saved to {EXCEL_FILE}")

# ====================
# Helper Function to Get Correct Ticker Symbol
# ====================

def get_symbol(ticker, exchange):
    """
    Returns the proper symbol for yfinance based on exchange.
    For NSE, suffix is .NS; for BSE, suffix is .BO.
    """
    if exchange == "NS":
        return f"{ticker}.NS"
    elif exchange == "BSE":
        return f"{ticker}.BO"
    else:
        return f"{ticker}.{exchange}"

# ====================
# Stock Data & Report Functions
# ====================

def fetch_stock_prices(ticker, exchange):
    """
    Fetch and display daily stock prices.
    Also saves a summary (high and low) to an Excel file.
    """
    symbol = get_symbol(ticker, exchange)
    stock = yf.Ticker(symbol)
    data = stock.history(period="1d")

    if not data.empty:
        opening_price = data['Open'].iloc[0]
        high_price = data['High'].iloc[-1]
        low_price = data['Low'].iloc[-1]
        closing_price = data['Close'].iloc[-1]

        print(f"Daily Stock Prices for {ticker}:")
        print(f"  Opening Price: {opening_price}")
        print(f"  High Price: {high_price}")
        print(f"  Low Price: {low_price}")
        print(f"  Closing Price: {closing_price}")

        save_to_excel([ticker, exchange, "1d", high_price, low_price])
    else:
        print(f"Error: Failed to retrieve daily data for {ticker}.")

def fetch_option_report(option, ticker, exchange):
    """
    Fetch and display the high and low stock prices over a given period.
    Saves the report data to an Excel file.
    """
    symbol = get_symbol(ticker, exchange)
    stock = yf.Ticker(symbol)
    data = stock.history(period=option)

    if not data.empty:
        high_price = data['High'].max()
        low_price = data['Low'].min()

        print(f"{option} Report for {ticker}:")
        print(f"  High Price: {high_price}")
        print(f"  Low Price: {low_price}")

        save_to_excel([ticker, exchange, option, high_price, low_price])
    else:
        print(f"Error: No data available for {ticker} in the {option} period.")

def plot_candlestick(option, ticker, exchange):
    """
    Create an interactive candlestick chart.
    """
    symbol = get_symbol(ticker, exchange)
    stock = yf.Ticker(symbol)
    data = stock.history(period=option)

    if not data.empty:
        fig = go.Figure(data=[go.Candlestick(
            x=data.index,
            open=data['Open'],
            high=data['High'],
            low=data['Low'],
            close=data['Close']
        )])
        fig.update_layout(title=f"{ticker} Candlestick Chart ({option})",
                          yaxis_title="Price")
        fig.show()
    else:
        print(f"Error: No data available for {ticker} in the {option} period.")

def plot_line_chart(option, ticker, exchange):
    """
    Create a line chart for the closing prices.
    """
    symbol = get_symbol(ticker, exchange)
    stock = yf.Ticker(symbol)
    data = stock.history(period=option)
    if not data.empty:
        fig = go.Figure(data=[go.Scatter(x=data.index, y=data['Close'],
                                          mode='lines', name='Closing Price')])
        fig.update_layout(title=f"{ticker} Line Chart - Closing Prices ({option})",
                          xaxis_title="Date", yaxis_title="Price")
        fig.show()
    else:
        print(f"Error: No data available for {ticker} in the {option} period for line chart.")

def plot_volume_chart(option, ticker, exchange):
    """
    Create a bar chart for the trading volume.
    """
    symbol = get_symbol(ticker, exchange)
    stock = yf.Ticker(symbol)
    data = stock.history(period=option)
    if not data.empty:
        fig = go.Figure(data=[go.Bar(x=data.index, y=data['Volume'], name='Volume')])
        fig.update_layout(title=f"{ticker} Volume Bar Chart ({option})",
                          xaxis_title="Date", yaxis_title="Volume")
        fig.show()
    else:
        print(f"Error: No data available for {ticker} in the {option} period for volume chart.")

def plot_area_chart(option, ticker, exchange):
    """
    Create an area chart for the closing prices.
    """
    symbol = get_symbol(ticker, exchange)
    stock = yf.Ticker(symbol)
    data = stock.history(period=option)
    if not data.empty:
        fig = go.Figure(data=[go.Scatter(x=data.index, y=data['Close'], fill='tozeroy',
                                          mode='lines', name='Closing Price')])
        fig.update_layout(title=f"{ticker} Area Chart - Closing Prices ({option})",
                          xaxis_title="Date", yaxis_title="Price")
        fig.show()
    else:
        print(f"Error: No data available for {ticker} in the {option} period for area chart.")

def detailed_report(option, ticker, exchange):
    """
    Generate a detailed report including moving averages, daily returns,
    descriptive statistics, and annualized volatility.
    """
    symbol = get_symbol(ticker, exchange)
    stock = yf.Ticker(symbol)
    data = stock.history(period=option)
    if not data.empty:
        data['MA20'] = data['Close'].rolling(window=20).mean()
        data['MA50'] = data['Close'].rolling(window=50).mean()
        data['Daily Return'] = data['Close'].pct_change()
        volatility = data['Daily Return'].std() * (252 ** 0.5)
        summary_stats = data.describe()
        print(f"Detailed Report for {ticker} ({option}):")
        print(summary_stats)
        print(f"Annualized Volatility: {volatility:.2%}")
        return data, summary_stats, volatility
    else:
        print(f"Error: No data available for {ticker} in the {option} period.")
        return None, None, None

# ====================
# Dash Web Dashboard
# ====================

app = Dash(__name__)
server = app.server  # for deploying

# Define available companies for each exchange
nse_companies = [
    {'label': 'INFY', 'value': 'INFY'},
    {'label': 'TCS', 'value': 'TCS'},
    {'label': 'WIPRO', 'value': 'WIPRO'},
]

bse_companies = [
    {'label': 'RELIANCE', 'value': 'RELIANCE'},
    {'label': 'ICICIBANK', 'value': 'ICICIBANK'},
    {'label': 'ZOMATO', 'value': 'ZOMATO'},
]

app.layout = html.Div([
    html.H1("Stock Analysis Dashboard", style={"textAlign": "center"}),
    html.Div([
        html.Label("Select Exchange:"),
        dcc.Dropdown(
            id="exchange-dropdown",
            options=[
                {'label': 'NSE', 'value': 'NS'},
                {'label': 'BSE', 'value': 'BSE'},
            ],
            value="NS",
            clearable=False
        )
    ], style={"width": "30%", "display": "inline-block", "padding": "10px"}),
    html.Div([
        html.Label("Select Ticker:"),
        dcc.Dropdown(
            id="ticker-dropdown",
            options=nse_companies,  # default options for NSE
            value="INFY",
            clearable=False
        )
    ], style={"width": "30%", "display": "inline-block", "padding": "10px"}),
    html.Div([
        html.Label("Select Period:"),
        dcc.Dropdown(
            id="period-dropdown",
            options=[
                {'label': '1d', 'value': '1d'},
                {'label': '1mo', 'value': '1mo'},
                {'label': '3mo', 'value': '3mo'},
                {'label': '6mo', 'value': '6mo'},
                {'label': '1y', 'value': '1y'},
            ],
            value="1mo",
            clearable=False
        )
    ], style={"width": "30%", "display": "inline-block", "padding": "10px"}),
    html.Br(),
    html.Button("Get Report", id="report-button", n_clicks=0),
    html.Br(), html.Br(),
    html.Div(id="report-output", style={"textAlign": "center", "fontSize": 18}),
    dcc.Graph(id="candlestick-chart"),
    dcc.Graph(id="line-chart"),
    dcc.Graph(id="volume-chart"),
    dcc.Graph(id="area-chart"),
    html.H2("Detailed Report Summary", style={"textAlign": "center"}),
    dash_table.DataTable(id="detailed-table",
                         style_table={'overflowX': 'auto'},
                         style_cell={'textAlign': 'center'}),
    html.Br()
])

# Callback to update the ticker options based on the selected exchange.
@app.callback(
    Output("ticker-dropdown", "options"),
    Input("exchange-dropdown", "value")
)
def update_ticker_options(exchange):
    if exchange == "NS":
        return nse_companies
    elif exchange == "BSE":
        return bse_companies
    else:
        return nse_companies

@app.callback(
    [Output("candlestick-chart", "figure"),
     Output("line-chart", "figure"),
     Output("volume-chart", "figure"),
     Output("area-chart", "figure"),
     Output("detailed-table", "data"),
     Output("detailed-table", "columns"),
     Output("report-output", "children")],
    [Input("report-button", "n_clicks")],
    [State("ticker-dropdown", "value"),
     State("exchange-dropdown", "value"),
     State("period-dropdown", "value")]
)
def update_dashboard(n_clicks, ticker, exchange, period):
    if n_clicks == 0:
        return {}, {}, {}, {}, [], [], ""

    data, summary_stats, volatility = detailed_report(period, ticker, exchange)
    if data is None:
        return {}, {}, {}, {}, [], [], "Error fetching data."

    candlestick_fig = go.Figure(data=[go.Candlestick(
        x=data.index,
        open=data['Open'],
        high=data['High'],
        low=data['Low'],
        close=data['Close']
    )])
    candlestick_fig.update_layout(title=f"{ticker} Candlestick Chart ({period})",
                                  yaxis_title="Price")

    line_fig = go.Figure(data=[go.Scatter(x=data.index, y=data['Close'],
                                          mode='lines', name='Closing Price')])
    line_fig.update_layout(title=f"{ticker} Line Chart - Closing Prices ({period})",
                           xaxis_title="Date", yaxis_title="Price")

    volume_fig = go.Figure(data=[go.Bar(x=data.index, y=data['Volume'], name='Volume')])
    volume_fig.update_layout(title=f"{ticker} Volume Bar Chart ({period})",
                             xaxis_title="Date", yaxis_title="Volume")

    area_fig = go.Figure(data=[go.Scatter(x=data.index, y=data['Close'],
                                          fill='tozeroy', mode='lines', name='Closing Price')])
    area_fig.update_layout(title=f"{ticker} Area Chart - Closing Prices ({period})",
                           xaxis_title="Date", yaxis_title="Price")

    summary_df = summary_stats.reset_index().rename(columns={"index": "Statistic"})
    table_data = summary_df.to_dict('records')
    table_columns = [{"name": col, "id": col} for col in summary_df.columns]

    report_text = f"{ticker} Detailed Report ({period}) - Annualized Volatility: {volatility:.2%}"

    return candlestick_fig, line_fig, volume_fig, area_fig, table_data, table_columns, report_text

if __name__ == "__main__":
    app.run_server(debug=True)


<IPython.core.display.Javascript object>